In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('vizualizações3').getOrCreate()

23/11/02 21:52:28 WARN Utils: Your hostname, rodrigo-Nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp8s0)
23/11/02 21:52:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/02 21:52:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_parquet = spark.read.parquet('/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_cities_skylines/total_video_publicado_semana/total_video_publicado_semana.parquet')
df_parquet.createOrReplaceTempView('TOTAL_PUBLICACOES')

In [8]:
spark.sql('SELECT * FROM TOTAL_PUBLICACOES WHERE NM_CANAL LIKE "C%"').show(truncate=False)

+----------------+----------------------------+------------+---------------+------------------------+
|SEMANA_TRADUZIDA|NM_CANAL                    |TOTAL_VIDEOS|DATA_PUBLICACAO|ID_CANAL                |
+----------------+----------------------------+------------+---------------+------------------------+
|Segunda-feira   |CANAL HAND - GAME WITH MUSIC|1           |2023-10-16     |UC7v51W2NoLFawizgcNuVBOA|
|Segunda-feira   |CANAL HAND - GAME WITH MUSIC|1           |2023-10-23     |UC7v51W2NoLFawizgcNuVBOA|
|Quarta-feira    |CANAL HAND - GAME WITH MUSIC|1           |2023-10-18     |UC7v51W2NoLFawizgcNuVBOA|
|Terça-feira     |CANAL HAND - GAME WITH MUSIC|1           |2023-10-17     |UC7v51W2NoLFawizgcNuVBOA|
|Sexta-feira     |CANAL HAND - GAME WITH MUSIC|1           |2023-10-20     |UC7v51W2NoLFawizgcNuVBOA|
|Sábado          |CANAL HAND - GAME WITH MUSIC|1           |2023-10-21     |UC7v51W2NoLFawizgcNuVBOA|
|Domingo         |CANAL HAND - GAME WITH MUSIC|1           |2023-10-22     |UC7v51

In [14]:
spark.sql('SELECT DATA_PUBLICACAO, sum(TOTAL_VIDEOS) FROM TOTAL_PUBLICACOES GROUP BY  DATA_PUBLICACAO').show()

+---------------+-----------------+
|DATA_PUBLICACAO|sum(TOTAL_VIDEOS)|
+---------------+-----------------+
|     2023-10-25|               23|
|     2023-10-16|               16|
|     2023-10-15|               10|
|     2023-10-18|               14|
|     2023-10-23|               24|
|     2023-10-21|               21|
|     2023-10-20|               13|
|     2023-10-19|               17|
|     2023-10-22|               15|
|     2023-10-17|               16|
|     2023-10-26|               22|
|     2023-10-24|               23|
|     2023-10-27|               21|
+---------------+-----------------+



In [17]:
spark.sql('SELECT COUNT(*) FROM TOTAL_PUBLICACOES where DATA_PUBLICACAO = "2023-10-25"  ' ).show(26)

+--------+
|count(1)|
+--------+
|      23|
+--------+



# Relação Comentários X Likes (proporção)

# Relação Visualizações X Likes (proporção)

# Dia em que os vídeos são Públicados com alta frequência

# Total de Tags X Vísualizações 

# Tamanho dos títulos do Vídeos X Vísualizações

# Total Víews por Vídeo

# A duração do vídeo é importante para visualizações e interação (curtidas/comentários)?